In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Constantes
BATCH_SIZE = 256
EPOCHS = 100
NEURONS = 64
CAPAS = 3
RES_ESPACIAL = 128
CHANNELS = 3

def load_data(path, batch_size, res=RES_ESPACIAL):
    return tf.keras.utils.image_dataset_from_directory(
        path,
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        batch_size=batch_size,
        image_size=(res, res),
        shuffle=True
    )

In [3]:
train_images = load_data('data/train_images/', BATCH_SIZE)
reduced_train_images = load_data('data/reduced_train_images/', BATCH_SIZE)
val_images = load_data('data/val_images/', BATCH_SIZE)

Found 185101 files belonging to 46 classes.
Found 37020 files belonging to 46 classes.
Found 46276 files belonging to 46 classes.


In [5]:
train_metadata = pd.read_csv('data/metadata_train_df.csv')
reduced_train_metadata = pd.read_csv('data/metadata_reduced_train_df.csv')
val_metadata = pd.read_csv('data/metadata_val_df.csv')
test_metadata = pd.read_csv('data/metadata_test_df.csv')

In [6]:
def preprocess(x, y):
    normalization_layer = tf.keras.layers.Rescaling(1./255)
    return tf.reshape(normalization_layer(x), (-1, RES_ESPACIAL, RES_ESPACIAL, CHANNELS)), y

train_images_preprocessed = train_images.map(preprocess)
reduced_train_images_preprocessed = reduced_train_images.map(preprocess)
val_images_preprocessed = val_images.map(preprocess)

In [7]:
input = tf.keras.Input(shape=(RES_ESPACIAL, RES_ESPACIAL, CHANNELS))
hidden = tf.keras.layers.Conv2D(NEURONS, 3, activation='relu')(input)
for i in range(CAPAS - 1):
    hidden = tf.keras.layers.Conv2D(NEURONS, 3, activation='relu')(hidden)
hidden = tf.keras.layers.Flatten()(hidden)
hidden = tf.keras.layers.Dense(NEURONS, activation='relu')(hidden)
output = tf.keras.layers.Dense(2, activation='softmax')(hidden)

model = tf.keras.Model(inputs=input, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 124, 124, 64)      36928     
                                                                 
 conv2d_2 (Conv2D)           (None, 122, 122, 64)      36928     
                                                                 
 flatten (Flatten)           (None, 952576)            0         
                                                                 
 dense (Dense)               (None, 64)                60964928  
                                                                 
 dense_1 (Dense)             (None, 2)                 130   